In [ ]:
import numpy as np
import pandas as pd
import os
from os import path
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
# project_root: 项目根目录
project_root = '/home/project/gas_anormaly_detection/restaurant/2testing/'
data_root = path.join(project_root, 'Dataset/')
weekdata_path = data_root+'raw_test/'


In [ ]:
#load all restaurant ID
with open(os.path.join(project_root,'all_restaurant_id.txt'), "r") as f:
    all_restaurant_id = f.readlines()
    all_restaurant_id = all_restaurant_id[0].split(" ")

In [ ]:
#load abnormal restaurant ID
with open(os.path.join(project_root,'abnormal_restaurant_id.txt'), "r") as f:
    abnormal_restaurant_id = f.readlines()
    abnormal_restaurant_id = abnormal_restaurant_id[0].split(" ")

In [ ]:
#save detection results
detect_results_root = path.join(project_root, 'detect_results/')
if not os.path.exists(detect_results_root):
    os.mkdir(detect_results_root)
    print("creat new data dir.")


# rule-based detecion

In [ ]:
week_data_list = os.listdir(weekdata_path)

In [ ]:
# 预处理后的1周地址
weekdata_afterrules_path = path.join(data_root, 'weekdata_affter_afterrules')
if not os.path.exists(weekdata_afterrules_path):
    os.mkdir(weekdata_afterrules_path)
    print("creat new data dir.")

In [ ]:
week_len=168
zero_consumption_anomalies = []
negative_consumption_anomalies = []
burty_consumption_anomalies = []

for f_name in tqdm(week_data_list):
    id_ = os.path.splitext(f_name)[0]

    week_data = pd.read_csv(os.path.join(weekdata_path, f_name))
    week_data = week_data.iloc[0,:].values

        # rule1: zero consumption anomalies
        if (abs(week_data)<0.001).sum()==week_len:
            zero_consumption_anomalies.append(id_)
            continue

        # rule2: negative consumption anomalies
        week_data_tep = week_data.copy()
        week_data_tep[week_data_tep>0.001]=0
        if np.nansum(week_data_tep)<0:
            negative_consumption_anomalies.append(id_)
            continue

        # rule3: burty consumption anomalies
        mean_usage = []
        for i in range(7):
            mean_usage.append(np.nanmax(week_data[i*24:i*24+24]))
        mean_usage = np.nanmean(mean_usage)

        if (week_data> max(5*mean_usage,10)).sum()>0:
            burty_consumption_anomalies.append(id_)
            continue
            
                 
    np.savetxt(path.join(weekdata_afterrules_path, f_name), week_data)
            
        
with open(path.join(detect_results_root, 'zero_consumption_anomalies.txt'), 'w') as f:
    f.write(' '.join([str(i) for i in zero_consumption_anomalies]))
with open(path.join(detect_results_root, 'negative_consumption_anomalies.txt'), 'w') as f:
    f.write(' '.join([str(i) for i in negative_consumption_anomalies]))            
with open(path.join(detect_results_root, 'burty_consumption_anomalies.txt'), 'w') as f:
    f.write(' '.join([str(i) for i in burty_consumption_anomalies]))
            
            